In [97]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [98]:
def create_graph(df) -> dict:
    """
    returns a didct with keys = node key, and values = edges (node_id, edge weight) 
    """
    tree = {}
    for _, row in df.iterrows():
        parent = row['Parent']
        child = row['Child']
        weight = row['t']

        # root node
        if pd.isna(parent): 
            continue
        # node already exists
        elif parent in tree: 
            tree[parent].append((child, weight))
        # node does not exists
        else:
            tree[parent] = [(child, weight)]
    return tree


def pd_z0():
    return max(np.random.normal(alpha_0, np.sqrt(variance_0)), 1)


def cpd(z, t):
    mean = gamma_0 + (alpha * t) + (beta * z) + (gamma * t * z)
    var = variance * t
    return max(np.random.normal(mean, np.sqrt(var)), 1)


def simulation(root: int, graph: dict, alpha0, variance0, alpha, beta, variance) -> dict:
    """
    BFS
    returns z value for all 407 nodes
    """
    gene_lengths = {}
    
    z0 = max(np.random.normal(alpha0, np.sqrt(variance0)), 1)
    queue = [(root, z0)]

    while queue:
        # handle the next element in queue
        node, z = queue.pop(0)
        
        # set node in results in genelength z
        gene_lengths[node] = z
        
        # get all children for the node
        children = graph.get(node, [])

        if not children:
            continue
        else:
            # for all the children for this node calculate their z
            # since we know it's a tree, only the parent can have influence
            for child in children:
                id, t = child
                t = round(t, 3)
                
                # draw a sample
                mean = (alpha * t) + (beta * z)
                var = variance * t
                
                    
                cpd_z = max(np.random.normal(mean, np.sqrt(var)), 1)
                #print(f"id:{id}, t:{t}, var:{var}, std:{np.sqrt(var)}, mean:{mean}, cpd:{cpd_z}")
                #if id == 205:
                #    print(var)
                #    print(mean)
                #    print(cpd_z)
                # append child to queue
                queue.append((id, cpd_z))

    return gene_lengths


def n_simulations(n, root, graph, alpha0, variance0, alpha, beta, variance) -> tuple:
    """
    Returns results for X=X1 ... X204 n times, and y=Z0 n times s
    """
    X = np.empty((n, 204))
    Z = np.empty((n, 203))
    y = np.empty(n)
    
    for i in range(n):
        results = simulation(root=root, graph=graph, alpha0=alpha0[i], variance0=variance0[i], alpha=alpha[i], beta=beta[i], variance=variance[i])
        
        # extract the first 204 values from the dictionary and add them to X as a row
        row_X = [results[key] for key in range(1, 205)]
        row_Z = [results[key] for key in range(205, 408)]
        
        X[i] = np.array(row_X)
        Z[i] = np.array(row_Z)
        
        
    return X, Z



def dfs_paths_with_cpd(graph, start_node, z, path=None):
    # returns all paths and the gene length at each node in the path
    if path is None:
        path = [(start_node, z)]

    if start_node not in graph:
        return [path]

    paths = []
    for child, t in graph[start_node]:
        if child not in [node for node, _ in path]:
            new_z = cpd(z, t)
            child_paths = dfs_paths_with_cpd(graph, child, new_z, path + [(child, new_z)])
            paths.extend(child_paths)

    return paths

In [99]:
# load data
tree_data = pd.read_csv('data/tree.csv')
genes_data = pd.read_csv('data/vert_genes.csv')

# remove root node, since we already have reference to it from else where. unless issue with t?
tree_data = tree_data[tree_data['Parent'].notna()]

# convert parent col to ints
tree_data['Parent'] = tree_data['Parent'].astype(int)

In [115]:
# gamma_0 + (alpha * time) (beta * parent gene len) + (gamma * time * parent gene len)

# Parameters
# for Z_0
size = 5000
alpha0 = np.random.normal(50000, 50, size=size)
variance0 = np.random.normal(5000, 10, size=size)

# for Z_i and X_i
alpha = np.random.normal(0, 0.5, size=size)
beta = np.random.normal(1, 0.5, size=size)
variance = np.random.normal(2500, 10, size=size)

# Settings
root = 407

# create a dict graphe
graph = create_graph(tree_data)

In [116]:
print(alpha0.shape)

(5000,)


In [117]:
# Create data
t0 = time.time()

X, Z = n_simulations(size, root, graph, alpha0, variance0, alpha, beta, variance)
runtime = time.time() - t0
print(f"{runtime:.10f} sec")

11.1088550091 sec


In [118]:
V = np.concatenate((X,Z),axis=1)
print(V)
#V = np.concatenate(X,Z,alpha0,variance0,alpha,beta,variance)

[[6.17393885e+03 6.19982591e+03 6.85236487e+03 ... 2.21817237e+04
  2.49987600e+04 4.99785999e+04]
 [1.66617795e+04 1.66808852e+04 1.75929181e+04 ... 3.28565512e+04
  3.48009465e+04 5.01124008e+04]
 [1.00000000e+00 2.19569460e+02 1.00000000e+00 ... 5.93190327e+02
  1.06469957e+03 5.00457284e+04]
 ...
 [1.39854334e+07 1.39854487e+07 1.02275579e+07 ... 4.49421394e+05
  3.28717607e+05 5.00245716e+04]
 [6.95480024e+04 6.97026027e+04 6.80635366e+04 ... 5.71896377e+04
  5.60733356e+04 4.99743051e+04]
 [1.63185145e+07 1.63187518e+07 1.18308934e+07 ... 4.75423467e+05
  3.44609483e+05 5.00207783e+04]]


In [119]:
# lin r
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regAlpha = LinearRegression().fit(V, alpha)
regBeta = LinearRegression().fit(V, beta)
regSigma = LinearRegression().fit(V, variance)

# Train results
yt_pred = regAlpha.predict(V)
train_r2 = regAlpha.score(V, alpha)
train_mse = mean_squared_error(alpha, yt_pred)

yt_pred1 = regBeta.predict(V)
yt_pred2 = regSigma.predict(V)
'''# Test results
y_pred = reg.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
test_mse = mean_squared_error(y_test, y_pred)'''

print(f"TRAIN. r2: {train_r2:.3f}, MSE: {train_mse:.3f}")
#print(f"TEST. r2: {test_r2:.3f}, MSE: {test_mse:.3f}")

TRAIN. r2: 0.192, MSE: 0.207


In [120]:
print(yt_pred)
print(alpha)

print(regBeta.score(V,beta))
print(regSigma.score(V,variance))
print((yt_pred1-beta))
print((yt_pred2,variance))
print(np.min(yt_pred2))
print(np.min(variance))

[ 2.44198439e-01 -3.85595290e-04 -2.67656462e-02 ... -4.62825492e-01
  2.26596117e-01  8.65992910e-02]
[-0.09824802  0.29408099  0.14650702 ...  0.00471274 -0.42163605
 -0.85698018]
0.9963042726961431
0.09064019245570931
[ 0.0090476   0.0103847  -0.0002175  ... -0.00441527 -0.00641005
  0.00483902]
(array([2500.963598  , 2498.30052594, 2498.7826872 , ..., 2497.88379795,
       2502.1648148 , 2499.51122527]), array([2505.50460715, 2503.70686183, 2497.91219724, ..., 2490.68477301,
       2505.96769832, 2506.51934738]))
2478.399855960714
2460.166710010255
